# Data Generation

## Import Packages

In [209]:
import numpy as np
import scipy
from scipy.stats import norm
import pandapower as pp
import pandas as pd
import pandapower.networks
import tensorflow as tf

## Input Data

In [222]:
case = pp.networks.case9()
number_of_bus = case.bus.shape[0]
baseMVA = 100
number_of_data = 2000

load = case.load.iloc[:,6:8]
load_bus_number = case.load.iloc[:,0]

Pgenerators = np.zeros((number_of_bus,1))
Pgenerators[case.gen.iloc[:,0],0] = -case.gen.iloc[:,4]/baseMVA

PQload = np.zeros((number_of_bus,2))
PQload[case.load.iloc[:,0],:] = case.load.iloc[:,6:8]/baseMVA



### NB FB TB R X PL QL PG

Data_form = np.hstack(((np.array(case.bus_geodata.iloc[:,0:2])),\
                       PQload,Pgenerators))

## Roulette Wheel Algorithm Using Normal-PDF to Generate Scenarios

In [234]:
Scenario = tf.zeros([number_of_data,Data_form.shape[0],Data_form.shape[1]])

for i in range(number_of_data):
    
    Scenario[i,:,:] = tf.Tensor(Data_form)
    scenario[i,load_bus_number,2:4] = norm.pff(np.random.rand(case.bus.shape[0],1),\
                                                              load,np.multiply(0.1,load))

TypeError: __init__() missing 2 required positional arguments: 'value_index' and 'dtype'